In [2]:
import os
import json
import numpy as np
from glob import glob
from PIL import Image
from numpy import argmax
from tqdm.auto import tqdm

from skimage.io import imread
from skimage.transform import resize

from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate, Reshape, Softmax
from tensorflow.keras.layers import Conv1D, Conv2D, Conv3D, AveragePooling3D, Conv2DTranspose
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
sess = tf.compat.v1.Session(config=config)
tf.random.set_seed(1234)

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2070 SUPER, pci bus id: 0000:09:00.0, compute capability: 7.5
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device



In [4]:
MODELS = [
    "model_test_new_preprocess_fine_tuning_resnet152v2.h5", 
    "model_test_new_preprocess_fine_tuning_densenet201.h5",
]
PREPROCESSORS = [
    tf.keras.applications.resnet_v2.preprocess_input,
    tf.keras.applications.densenet.preprocess_input
]

In [5]:
cwd = "/Workspace/Shared"
data_path = os.path.join(cwd, 'Development_Dataset', 'Training', "Bipbip", "Mais", "Images", "*")
IMG_H = 768
IMG_W = 1024

# Load the pre-trained models

In [6]:
def load_all_models(models):
    return [
        print("loading %s"%model_name) or load_model(model_name, compile=False)
        for model_name in models
    ]

In [7]:
trained_models = load_all_models(MODELS)

loading model_test_new_preprocess_fine_tuning_resnet152v2.h5
loading model_test_new_preprocess_fine_tuning_densenet201.h5


# Load the Data

In [8]:
def rle_encode(img):
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def read_rgb_mask(img_path):
    mask_img = Image.open(img_path)
    mask_img = mask_img.resize([IMG_W, IMG_H], resample=Image.NEAREST)
    mask_arr = np.array(mask_img)

    new_mask_arr = np.zeros(mask_arr.shape[:2], dtype=mask_arr.dtype)

    # Use RGB dictionary in 'RGBtoTarget.txt' to convert RGB to target
    new_mask_arr[np.where(np.all(mask_arr == [216, 124, 18], axis=-1))] = 0
    new_mask_arr[np.where(np.all(mask_arr == [255, 255, 255], axis=-1))] = 1
    new_mask_arr[np.where(np.all(mask_arr == [216, 67, 82], axis=-1))] = 2
    return new_mask_arr

In [9]:
def load_image(path):
    return resize(
        imread(path)[:,:,:3], 
        (IMG_H, IMG_W), 
        mode='constant', 
        preserve_range=True
    )

def load_mask(path):
    mask = read_rgb_mask(path.replace("jpg", "png").replace("Images", "Masks"))
    mask = np.expand_dims(mask, -1)
    return np.float32(mask)

In [9]:
filenames = glob(data_path)

In [10]:
masks = np.array([
    load_mask(path)
    for path in tqdm(filenames, desc="Loading masks")
])

In [11]:
imgs = np.array([
    load_image(path)
    for path in tqdm(filenames, desc="Loading images")
])

# Training data based on the models' predictions

In [12]:
%%time
preds = [
    model.predict(preprocessor(imgs), batch_size=1)
    for (model, preprocessor) in tqdm(zip(trained_models, PREPROCESSORS), desc="Computing the predictions")
] 


CPU times: user 21.1 s, sys: 2min 2s, total: 2min 23s
Wall time: 4min 27s


In [13]:
x = dict(list(zip(["img"] + MODELS, [imgs, *preds])))
y = to_categorical(masks)
x_pred = preds

In [14]:
for k, v in x.items():
    print(k, v.shape)

img (90, 768, 1024, 3)
model_test_new_preprocess_fine_tuning_resnet152v2.h5 (90, 768, 1024, 3)
model_test_new_preprocess_fine_tuning_densenet201.h5 (90, 768, 1024, 3)


In [15]:
y.shape

(90, 768, 1024, 3)

# Build the ensamble

In [55]:
def compute_segmentation_masks_count(img_path):
        '''
        img_path: path to the mask file
        Returns the numpy array containing target values
        '''

        mask_img = Image.open(img_path)
        mask_img = mask_img.resize([IMG_H, IMG_W], resample=Image.NEAREST)
        mask_arr = np.array(mask_img)
        
        new_mask_arr = np.zeros(mask_arr.shape[:2], dtype=mask_arr.dtype)
    
        # Use RGB dictionary in 'RGBtoTarget.txt' to convert RGB to target
        new_mask_arr[np.where(np.all(mask_arr == [216, 124, 18], axis=-1))] = 0
        new_mask_arr[np.where(np.all(mask_arr == [255, 255, 255], axis=-1))] = 1
        new_mask_arr[np.where(np.all(mask_arr == [216, 67, 82], axis=-1))] = 2
        
        unique, counts = np.unique(new_mask_arr, return_counts=True)
        while len(counts) < 3:
            counts = np.append(counts, [1])
        return counts

def compute_weight_unbalance_mean(filenames):
        counts = np.array([0, 0, 0], dtype=np.float64)
        for img_path in filenames:
            count = compute_segmentation_masks_count(img_path.replace("jpg", "png").replace("Images", "Masks"))
            count_norm = count / np.linalg.norm(count)
            count_norm[1] = count_norm[0] / count_norm[1]
            count_norm[2] = count_norm[0] / count_norm[2]
            counts += count_norm
        classes = [0, 1, 2]
        return dict(zip(classes, counts/len(filenames)))
    
def compute_weight_unbalance_matrix(filenames):
        counts = []
        for img_path in filenames:
            count = compute_segmentation_masks_count(img_path.replace("jpg", "png").replace("Images", "Masks"))
            count_norm = count / np.linalg.norm(count)
            count_norm[1] = count_norm[0] / count_norm[1]
            count_norm[2] = count_norm[0] / count_norm[2]
            counts.append(count_norm)
        return np.asarray(counts)
    
sample_weights = compute_weight_unbalance_matrix(filenames)
print(sample_weights)
from itertools import cycle
sample_weights_iterator = cycle(sample_weights)

[[9.91388453e-01 7.59034180e+00 1.04802129e+02]
 [9.97775208e-01 2.55845070e+01 1.84529783e+01]
 [9.88744814e-01 7.06098058e+00 1.87679366e+01]
 [9.98590671e-01 2.27796510e+01 3.33793528e+01]
 [9.92736292e-01 8.68022649e+00 2.65822001e+01]
 [9.98286903e-01 1.74404573e+01 8.23728016e+01]
 [9.87852577e-01 6.35760677e+00 4.95879386e+02]
 [9.82198355e-01 1.37413861e+01 5.65403482e+00]
 [9.86196362e-01 6.49919272e+00 1.48822007e+01]
 [9.89551853e-01 8.06691706e+00 1.30615118e+01]
 [9.92396201e-01 9.33784234e+00 1.59835021e+01]
 [9.91751628e-01 3.35753330e+01 7.95153639e+00]
 [9.88449424e-01 6.97036502e+00 1.84881475e+01]
 [9.98624841e-01 2.18205006e+01 3.90507555e+01]
 [9.98274338e-01 1.71348163e+01 1.35698419e+02]
 [9.94010328e-01 9.56067652e+00 2.95179045e+01]
 [9.60956715e-01 4.73559138e+00 5.10852678e+00]
 [9.94691254e-01 9.73227934e+00 8.30727744e+01]
 [9.94327930e-01 1.14443255e+01 1.62089096e+01]
 [9.98842643e-01 2.29785772e+01 4.85155255e+01]
 [9.73072688e-01 1.22723314e+01 4.495982

In [64]:
# Here we define the intersection over union for each class in the batch.
# Then we compute the final iou as the mean over classes

def weightedLoss(originalLossFunc):

    def lossFunc(true, pred):

        axis = -1
        classSelectors = K.argmax(true, axis=axis)
        weightsList = next(sample_weights_iterator)
        
        classSelectors = [K.equal(tf.cast(i, tf.int64), classSelectors) for i in range(len(weightsList))]

        classSelectors = [K.cast(x, K.floatx()) for x in classSelectors]
        
        weights = [sel * w for sel,w in zip(classSelectors, weightsList)] 

        weightMultiplier = weights[0]
        for i in range(1, len(weights)):
            weightMultiplier = weightMultiplier + weights[i]

        loss = originalLossFunc(true,pred) 
        loss = loss * weightMultiplier

        return loss
    return lossFunc

def meanIoU(y_true, y_pred):
    # get predicted class from softmax
    y_pred = tf.expand_dims(tf.argmax(y_pred, -1), -1)
    y_true = tf.expand_dims(tf.argmax(y_true, -1), -1)

    per_class_iou = []

    for i in range(1,3): # exclude the background class 0
        # Get prediction and target related to only a single class (i)
        class_pred = tf.cast(tf.where(y_pred == i, 1, 0), tf.float32)
        class_true = tf.cast(tf.where(y_true == i, 1, 0), tf.float32)
        intersection = tf.reduce_sum(class_true * class_pred)
        union = tf.reduce_sum(class_true) + tf.reduce_sum(class_pred) - intersection
    
        iou = (intersection + 1e-7) / (union + 1e-7)
        per_class_iou.append(iou)

    return tf.reduce_mean(per_class_iou)

In [65]:
inputs = [tf.keras.layers.Input([IMG_H, IMG_W, 3]), tf.keras.layers.Input([IMG_H, IMG_W, 3])]
merge = Concatenate(axis=3)(inputs)
hidden = tf.keras.layers.Dense(6, activation='relu')(merge)
output = tf.keras.layers.Dense(3, activation='softmax')(hidden)

In [66]:
stacked_model = Model(inputs=inputs, outputs=output)
stacked_model.compile(
    optimizer=tf.keras.optimizers.Nadam(learning_rate=1e-4), 
    loss=weightedLoss(lambda x, y: K.categorical_crossentropy(x, y, axis=3)), 
    metrics=['accuracy', meanIoU]
)
stacked_model.summary()
print(stacked_model.outputs[0].shape)

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           [(None, 768, 1024, 3 0                                            
__________________________________________________________________________________________________
input_22 (InputLayer)           [(None, 768, 1024, 3 0                                            
__________________________________________________________________________________________________
concatenate_10 (Concatenate)    (None, 768, 1024, 6) 0           input_21[0][0]                   
                                                                 input_22[0][0]                   
__________________________________________________________________________________________________
dense_15 (Dense)                (None, 768, 1024, 6) 42          concatenate_10[0][0]      

# Fit the ensamble

In [67]:
callbacks = []

rlp_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor="loss", min_delta=0.001, patience=3, factor=0.1)
callbacks.append(rlp_callback)

es_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
callbacks.append(es_callback)

stacked_model.fit(
    preds, y, 
    epochs=300,
    batch_size=1
)

Epoch 1/300
90/90 [==============================] - 2s 17ms/step - loss: 5.9263 - accuracy: 0.0851 - meanIoU: 0.0426
Epoch 2/300
90/90 [==============================] - 1s 17ms/step - loss: 5.8151 - accuracy: 0.0851 - meanIoU: 0.0426
Epoch 3/300
90/90 [==============================] - 2s 17ms/step - loss: 5.7048 - accuracy: 0.0851 - meanIoU: 0.0426
Epoch 4/300
90/90 [==============================] - 1s 16ms/step - loss: 5.5985 - accuracy: 0.0851 - meanIoU: 0.0426
Epoch 5/300
90/90 [==============================] - 1s 17ms/step - loss: 5.4959 - accuracy: 0.0851 - meanIoU: 0.0426
Epoch 6/300
90/90 [==============================] - 1s 17ms/step - loss: 5.3969 - accuracy: 0.0851 - meanIoU: 0.0426
Epoch 7/300
90/90 [==============================] - 1s 16ms/step - loss: 5.3005 - accuracy: 0.0851 - meanIoU: 0.0426
Epoch 8/300
90/90 [==============================] - 1s 16ms/step - loss: 5.2059 - accuracy: 0.0851 - meanIoU: 0.0426
Epoch 9/300
90/90 [==============================] - 1s 

90/90 [==============================] - 1s 16ms/step - loss: 1.6317 - accuracy: 0.9772 - meanIoU: 0.7941
Epoch 70/300
90/90 [==============================] - 1s 16ms/step - loss: 1.5999 - accuracy: 0.9774 - meanIoU: 0.7947
Epoch 71/300
90/90 [==============================] - 1s 16ms/step - loss: 1.5688 - accuracy: 0.9775 - meanIoU: 0.7954
Epoch 72/300
90/90 [==============================] - 1s 16ms/step - loss: 1.5385 - accuracy: 0.9776 - meanIoU: 0.7959
Epoch 73/300
90/90 [==============================] - 1s 16ms/step - loss: 1.5085 - accuracy: 0.9777 - meanIoU: 0.7964
Epoch 74/300
90/90 [==============================] - 1s 16ms/step - loss: 1.4795 - accuracy: 0.9778 - meanIoU: 0.7969
Epoch 75/300
90/90 [==============================] - 1s 16ms/step - loss: 1.4507 - accuracy: 0.9779 - meanIoU: 0.7975
Epoch 76/300
90/90 [==============================] - 1s 16ms/step - loss: 1.4228 - accuracy: 0.9780 - meanIoU: 0.7979
Epoch 77/300
90/90 [==============================] - 1s 16ms

90/90 [==============================] - 1s 16ms/step - loss: 0.5600 - accuracy: 0.9783 - meanIoU: 0.7987
Epoch 138/300
90/90 [==============================] - 1s 16ms/step - loss: 0.5555 - accuracy: 0.9783 - meanIoU: 0.7986
Epoch 139/300
90/90 [==============================] - 1s 16ms/step - loss: 0.5512 - accuracy: 0.9783 - meanIoU: 0.7985
Epoch 140/300
90/90 [==============================] - 1s 16ms/step - loss: 0.5471 - accuracy: 0.9782 - meanIoU: 0.7984
Epoch 141/300
90/90 [==============================] - 1s 16ms/step - loss: 0.5432 - accuracy: 0.9782 - meanIoU: 0.7981
Epoch 142/300
90/90 [==============================] - 1s 17ms/step - loss: 0.5393 - accuracy: 0.9782 - meanIoU: 0.7981
Epoch 143/300
90/90 [==============================] - 1s 16ms/step - loss: 0.5357 - accuracy: 0.9781 - meanIoU: 0.7979
Epoch 144/300
90/90 [==============================] - 1s 16ms/step - loss: 0.5321 - accuracy: 0.9781 - meanIoU: 0.7977
Epoch 145/300
90/90 [==============================] -

# Compute predictions

In [10]:
from skimage.transform import resize
from skimage.io import imread
from sklearn.model_selection import train_test_split
from PIL import Image
import json

def rle_encode(img):
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

test_filenames = glob(os.path.join(cwd, 'Development_Dataset', 'Test_Dev', "**", "**", "**", "*"))

submission_dict={}

imgs = np.array([
    load_image(path)
    for path in tqdm(test_filenames, desc="Loading images")
])

predictions = [
    stacked_model.predict(preprocessor(imgs), batch_size=1)
]

In [12]:
for pred, path in zip(predictions, test_filenames):

    pred = tf.image.resize(pred, [1536, 2048], method='nearest')
    mask_arr = tf.argmax(pred, -1).numpy()

    path_metadata = path.split("/")[5:]
    
    team = path_metadata[0]
    crop = path_metadata[1]
    img_name = path_metadata[3].replace(".jpg","").replace(".png","")
    
    submission_dict[img_name] = {}
    submission_dict[img_name]['shape'] = mask_arr.shape
    submission_dict[img_name]['team'] = team
    submission_dict[img_name]['crop'] = crop
    submission_dict[img_name]['segmentation'] = {}

    rle_encoded_crop = rle_encode(mask_arr == 1)
    rle_encoded_weed = rle_encode(mask_arr == 2)

    submission_dict[img_name]['segmentation']['crop'] = rle_encoded_crop
    submission_dict[img_name]['segmentation']['weed'] = rle_encoded_weed

with open('./predictions/submission.json', 'w') as f:
        json.dump(submission_dict, f)